In [1]:

using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials

In [2]:

# import conventions
include("conventions.jl")
  using .conventions: big_endian
# import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate

Load quantum gates constructor


In [ ]:
#include("lib_qcircuit/MK_qcircuit.jl")


In [3]:
include("quantum_circuit.jl")
using ..quantum_circuit: qc_initialize, init_register
#using ..quantum_circuit: qc_initialize, init_register, print_initstate

Load quantum gates constructor
Load quantum gates constructor


In [4]:
Ugate2 = [0 1; 1 0]
qubit_control = 0
qubit_target = 1
nqubits = 2
err_tol = 1e-8


1.0e-8

In [6]:
register, q_states, n_bas, n_dim, q_order  = init_register(nqubits)

([0.0, 0.0, 0.0, 0.0], [0.0 0.0; 0.0 1.0; 1.0 0.0; 1.0 1.0], 2, 4, "big-endian")

In [7]:
qc = qc_initialize(nqubits)

Main.quantum_circuit.qc_initstruct(2, "big-endian", 2, 4, [1.0, 0.0, 0.0, 0.0], [0.0 0.0; 0.0 1.0; 1.0 0.0; 1.0 1.0])

In [8]:
qc.state_vector

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [11]:
qc.q_states
qc.n_qubits

2

In [31]:
# print the initial state of the quantum register
function print_initstate(qc)
    # print the initial state of the quantum register
    # qc::qc_initstruct: quantum register
    # return: print the initial state of the quantum register
    println("The initial state of the quantum register is: ")
    println(qc.state_vector)
    # print the initial state of the quantum register with the quantum 
    # states in the computational basis
    println("The initial state of the quantum register with the 
    quantum states in the computational basis is: ")
    q_table = zeros(Int, qc.n_dim, qc.n_qubits)
    for iq=1:qc.n_qubits
        for i in 1:qc.n_dim
            q_table[i,iq] = trunc(Int, qc.q_states[i,iq])
        end # end for
    end # end for


    for i in 1:qc.n_dim
        println( qc.q_states[i], " | ", string(q_table[i,:]), ">")
    end # end for
    #show(stdout, "text/plain", [qc.state_vector trunc(Int,qc.q_states)])
end # end print_initstate

print_initstate (generic function with 1 method)

In [32]:
print_initstate(qc)

The initial state of the quantum register is: 
[1.0, 0.0, 0.0, 0.0]
The initial state of the quantum register with the 
    quantum states in the computational basis is: 
0.0 | [0, 0]>
0.0 | [0, 1]>
1.0 | [1, 0]>
1.0 | [1, 1]>


In [ ]:
Base.@kwdef mutable struct qc_initstruct
    # Initialize the quantum register
    n::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector::Array{Float64,1}
    q_states::Array{Float64,2}
end # end qc_initialize

In [ ]:

function qc_initialize(n::Int64, 
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol,
    q_order::String="big-endian")
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    ## notes:
    # the minimum number of qubits is 2 

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

    return qc_initstruct(n, q_order, n_bas, n_dim, state_vector, q_states)
end # end qc_initialize  

In [ ]:
qc = qc_initialize(2)

In [ ]:
qc.state_vector